In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf

import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, classification_report
%matplotlib inline

In [2]:
df = pd.read_csv('flight(raw data).csv')

In [3]:
df.shape

(10347, 21)

In [4]:
df.drop(['icao24', 'squawk', 'radar', 'time', 'departure', 'destination', 'on_ground', 'airline_icao'], axis=1, inplace=True)
df.shape

(10347, 13)

In [5]:
# MENGHAPUS ALTITUDE BERNILAI 0
df = df[df['altitude'] != 0]
df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign
0,1,9/26/2022,0.5088,101.4452,181,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276
1,2,9/26/2022,0.5028,101.4451,180,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276
2,3,9/26/2022,0.5004,101.4451,179,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276
3,4,9/26/2022,0.4986,101.4451,179,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276
4,5,9/26/2022,0.4964,101.4451,179,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276


In [6]:
# MEMBERI LABEL UNTUK MASING-MASING PENERBANGAN
df['label'] = (df['registration'] != df['registration'].shift(1)) | (df['callsign'] != df['callsign'].shift(1))
df['label'] = df['label'].cumsum()
df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label
0,1,9/26/2022,0.5088,101.4452,181,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1
1,2,9/26/2022,0.5028,101.4451,180,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1
2,3,9/26/2022,0.5004,101.4451,179,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1
3,4,9/26/2022,0.4986,101.4451,179,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1
4,5,9/26/2022,0.4964,101.4451,179,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1


In [7]:
# MENGUBAH LATITUDE DAN LOGNITUDE KE RADIAN
df['latitude_rad'] = np.radians(df['latitude'])
df['longitude_rad'] = np.radians(df['longitude'])
df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label,latitude_rad,longitude_rad
0,1,9/26/2022,0.5088,101.4452,181,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008880,1.770553
1,2,9/26/2022,0.5028,101.4451,180,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008776,1.770551
2,3,9/26/2022,0.5004,101.4451,179,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008734,1.770551
3,4,9/26/2022,0.4986,101.4451,179,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008702,1.770551
4,5,9/26/2022,0.4964,101.4451,179,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1,0.008664,1.770551


In [8]:
# MENGUBAH ALTITUDE KE METER
df['altitude_meter'] = df['altitude'] * 0.3048
df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label,latitude_rad,longitude_rad,altitude_meter
0,1,9/26/2022,0.5088,101.4452,181,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008880,1.770553,350.52
1,2,9/26/2022,0.5028,101.4451,180,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008776,1.770551,312.42
2,3,9/26/2022,0.5004,101.4451,179,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008734,1.770551,297.18
3,4,9/26/2022,0.4986,101.4451,179,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008702,1.770551,289.56
4,5,9/26/2022,0.4964,101.4451,179,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1,0.008664,1.770551,274.32


In [9]:
# MENGGANTI HEADING MENJADI 18 DAN 36
df['heading'] = np.where(np.isclose(df['heading'], 360, atol=9), 36, np.where(np.isclose(df['heading'], 180, atol=9), 18, df['heading'])).astype(int)

df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label,latitude_rad,longitude_rad,altitude_meter
0,1,9/26/2022,0.5088,101.4452,18,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008880,1.770553,350.52
1,2,9/26/2022,0.5028,101.4451,18,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008776,1.770551,312.42
2,3,9/26/2022,0.5004,101.4451,18,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008734,1.770551,297.18
3,4,9/26/2022,0.4986,101.4451,18,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008702,1.770551,289.56
4,5,9/26/2022,0.4964,101.4451,18,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1,0.008664,1.770551,274.32


In [10]:
def update_heading(heading):
    if heading in range(1, 10) or heading in range(300, 361):
        return 36
    else:
        return heading

# Assuming your DataFrame is named df
df['heading'] = df['heading'].apply(update_heading)

print(df)


       flight_id    tanggal  latitude  longitude  heading  altitude  \
0              1  9/26/2022    0.5088   101.4452       18      1150   
1              2  9/26/2022    0.5028   101.4451       18      1025   
2              3  9/26/2022    0.5004   101.4451       18       975   
3              4  9/26/2022    0.4986   101.4451       18       950   
4              5  9/26/2022    0.4964   101.4451       18       900   
...          ...        ...       ...        ...      ...       ...   
10280      10281  10/5/2022    0.4353   101.4437       36       400   
10281      10282  10/5/2022    0.4397   101.4439       36       325   
10282      10283  10/5/2022    0.4420   101.4440       36       275   
10283      10284  10/5/2022    0.4443   101.4440       36       225   
10284      10285  10/5/2022    0.4507   101.4442       36       125   

       ground_speed aircraft_code registration number airline_iata  \
0               140          B739       PK-LGR  JT276        JT276   
1      

In [11]:
import numpy as np

# Haversine function
def haversine(lat1, lon1, heading, R):
    lat2 = np.where(heading == 18, 0.008154, np.where(heading == 36, 0.007877, np.nan))
    lon2 = np.where(heading == 18, 1.770544, np.where(heading == 36, 1.770536, np.nan))

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Constants
R = 6371000  # Radius of the Earth

# Calculate Haversine distance for each row
df['haversine'] = haversine(df['latitude_rad'], df['longitude_rad'], df['heading'], R)

# Display the DataFrame with the new 'haversine' column
df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label,latitude_rad,longitude_rad,altitude_meter,haversine
0,1,9/26/2022,0.5088,101.4452,18,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008880,1.770553,350.52,4627.180494
1,2,9/26/2022,0.5028,101.4451,18,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008776,1.770551,312.42,3959.926594
2,3,9/26/2022,0.5004,101.4451,18,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008734,1.770551,297.18,3693.076943
3,4,9/26/2022,0.4986,101.4451,18,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008702,1.770551,289.56,3492.941526
4,5,9/26/2022,0.4964,101.4451,18,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1,0.008664,1.770551,274.32,3248.334158


In [12]:
# RUMUS ELEVASI
df['elevation'] = np.arctan(df['altitude_meter'] / df['haversine'])
df['elevation'] = np.degrees(df['elevation'])


df.head()

,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,number,airline_iata,vertical_speed,callsign,label,latitude_rad,longitude_rad,altitude_meter,haversine,elevation
0,1,9/26/2022,0.5088,101.4452,18,1150,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008880,1.770553,350.52,4627.180494,4.332019
1,2,9/26/2022,0.5028,101.4451,18,1025,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008776,1.770551,312.42,3959.926594,4.511029
2,3,9/26/2022,0.5004,101.4451,18,975,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008734,1.770551,297.18,3693.076943,4.600649
3,4,9/26/2022,0.4986,101.4451,18,950,140,B739,PK-LGR,JT276,JT276,-768,LNI276,1,0.008702,1.770551,289.56,3492.941526,4.738905
4,5,9/26/2022,0.4964,101.4451,18,900,141,B739,PK-LGR,JT276,JT276,-896,LNI276,1,0.008664,1.770551,274.32,3248.334158,4.827143


In [13]:
df['diff_vs'] = df['vertical_speed'].abs().diff().abs()
df['diff_vs'].fillna(0, inplace=True)

In [14]:
def iterate_and_add_distance(df):
    distances = []

    for label, group in df.groupby('label'):
        haversine_values = group['haversine'].tolist()

        for i in range(len(haversine_values) - 1):
            if haversine_values[i] > haversine_values[i + 1]:
                distances.append(haversine_values[i])
            else:
                distances.append(haversine_values[i])
                break  # Stop the loop after finding the minimum value

        # Append NaN for the remaining values
        for _ in range(i + 1, len(haversine_values)):
            distances.append(np.nan)

    # Create a new 'distance' column in the original DataFrame
    df['distance'] = distances

# Assuming your DataFrame is named df
iterate_and_add_distance(df)

# Now, df contains a new column named 'distance'
print(df)

       flight_id    tanggal  latitude  longitude  heading  altitude  \
0              1  9/26/2022    0.5088   101.4452       18      1150   
1              2  9/26/2022    0.5028   101.4451       18      1025   
2              3  9/26/2022    0.5004   101.4451       18       975   
3              4  9/26/2022    0.4986   101.4451       18       950   
4              5  9/26/2022    0.4964   101.4451       18       900   
...          ...        ...       ...        ...      ...       ...   
10280      10281  10/5/2022    0.4353   101.4437       36       400   
10281      10282  10/5/2022    0.4397   101.4439       36       325   
10282      10283  10/5/2022    0.4420   101.4440       36       275   
10283      10284  10/5/2022    0.4443   101.4440       36       225   
10284      10285  10/5/2022    0.4507   101.4442       36       125   

       ground_speed aircraft_code registration number  ... vertical_speed  \
0               140          B739       PK-LGR  JT276  ...           -

In [15]:
df.dropna(subset=['distance'], inplace=True)

# Now, df is updated with NaN rows removed
print(df)

       flight_id    tanggal  latitude  longitude  heading  altitude  \
0              1  9/26/2022    0.5088   101.4452       18      1150   
1              2  9/26/2022    0.5028   101.4451       18      1025   
2              3  9/26/2022    0.5004   101.4451       18       975   
3              4  9/26/2022    0.4986   101.4451       18       950   
4              5  9/26/2022    0.4964   101.4451       18       900   
...          ...        ...       ...        ...      ...       ...   
10279      10280  10/5/2022    0.4307   101.4436       36       475   
10280      10281  10/5/2022    0.4353   101.4437       36       400   
10281      10282  10/5/2022    0.4397   101.4439       36       325   
10282      10283  10/5/2022    0.4420   101.4440       36       275   
10283      10284  10/5/2022    0.4443   101.4440       36       225   

       ground_speed aircraft_code registration number  ... vertical_speed  \
0               140          B739       PK-LGR  JT276  ...           -

In [16]:
transformation = df.groupby('label').first().reset_index()

# Resetting index if needed
transformation.reset_index(drop=True, inplace=True)
transformation.head()

,label,flight_id,tanggal,latitude,longitude,heading,altitude,ground_speed,aircraft_code,registration,...,airline_iata,vertical_speed,callsign,latitude_rad,longitude_rad,altitude_meter,haversine,elevation,diff_vs,distance
0,1,1,9/26/2022,0.5088,101.4452,18,1150,140,B739,PK-LGR,...,JT276,-768,LNI276,0.008880,1.770553,350.52,4627.180494,4.332019,0.0,4627.180494
1,2,120,9/26/2022,0.4119,101.4431,36,975,158,B739,PK-LHQ,...,JT140,-704,LNI140,0.007189,1.770516,297.18,4385.010108,3.877109,128.0,4385.010108
2,3,191,9/26/2022,0.4152,101.4433,36,925,144,A320,PK-SJU,...,IU837,-640,SJV837,0.007247,1.770520,281.94,4017.594066,4.014226,512.0,4017.594066
3,4,260,9/26/2022,0.4138,101.4432,36,925,140,A320,PK-GLY,...,QG938,-640,CTV938,0.007222,1.770518,281.94,4173.509579,3.864725,640.0,4173.509579
4,5,330,9/26/2022,0.4139,101.4432,36,925,121,AT76,PK-WJP,...,IW1243,-576,WON1243,0.007224,1.770518,281.94,4162.394369,3.875014,128.0,4162.394369


In [17]:
transformation.drop(['flight_id', 'latitude', 'longitude', 'altitude', 'number', 'airline_iata','latitude_rad', 'longitude_rad', 'altitude_meter', 'haversine', 'elevation', 'distance', 'vertical_speed', 'ground_speed', 'diff_vs'], axis=1, inplace=True)

In [18]:
transformation.head()

,label,tanggal,heading,aircraft_code,registration,callsign
0,1,9/26/2022,18,B739,PK-LGR,LNI276
1,2,9/26/2022,36,B739,PK-LHQ,LNI140
2,3,9/26/2022,36,A320,PK-SJU,SJV837
3,4,9/26/2022,36,A320,PK-GLY,CTV938
4,5,9/26/2022,36,AT76,PK-WJP,WON1243


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1540 entries, 0 to 10283
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   flight_id       1540 non-null   int64  
 1   tanggal         1540 non-null   object 
 2   latitude        1540 non-null   float64
 3   longitude       1540 non-null   float64
 4   heading         1540 non-null   int64  
 5   altitude        1540 non-null   int64  
 6   ground_speed    1540 non-null   int64  
 7   aircraft_code   1540 non-null   object 
 8   registration    1540 non-null   object 
 9   number          1540 non-null   object 
 10  airline_iata    1540 non-null   object 
 11  vertical_speed  1540 non-null   int64  
 12  callsign        1540 non-null   object 
 13  label           1540 non-null   int32  
 14  latitude_rad    1540 non-null   float64
 15  longitude_rad   1540 non-null   float64
 16  altitude_meter  1540 non-null   float64
 17  haversine       1540 non-null   float

In [20]:
import pandas as pd

# Calculate average elevation for each label in df
avg_elevation_df = df.groupby('label')['elevation'].mean().reset_index()
avg_elevation_df.rename(columns={'elevation': 'avg_elevation'}, inplace=True)

# Merge with the transformation DataFrame and update transformation in-place
transformation = pd.merge(transformation, avg_elevation_df, on='label', how='left')

# Display the merged DataFrame
print(transformation)

     label    tanggal  heading aircraft_code registration callsign  \
0        1  9/26/2022       18          B739       PK-LGR   LNI276   
1        2  9/26/2022       36          B739       PK-LHQ   LNI140   
2        3  9/26/2022       36          A320       PK-SJU   SJV837   
3        4  9/26/2022       36          A320       PK-GLY   CTV938   
4        5  9/26/2022       36          AT76       PK-WJP  WON1243   
..     ...        ...      ...           ...          ...      ...   
126    127  10/4/2022       36          A320       PK-LUQ  BTK7065   
127    128  10/5/2022       36          A320       PK-GQA   CTV934   
128    129  10/5/2022       36          AT76       PK-WHI  WON1231   
129    130  10/5/2022       36          B739       PK-LHS   LNI239   
130    131  10/5/2022       36          B738       PK-GMD   GIA172   

     avg_elevation  
0         5.818741  
1         6.300770  
2         8.316118  
3         5.656915  
4         7.655067  
..             ...  
126       5.

In [21]:
# Calculating the mean elevation for each label in the original dataframe
avg_altitudes_m = df.groupby('label')['altitude_meter'].mean().reset_index()

# Merging the 'transformation' dataframe with the calculated mean elevations
transformation = pd.merge(transformation, avg_altitudes_m, on='label', how='left')

# Renaming the column to 'avg_elevation'
transformation.rename(columns={'altitude_meter': 'avg_altitude_m'}, inplace=True)


transformation.head()


,label,tanggal,heading,aircraft_code,registration,callsign,avg_elevation,avg_altitude_m
0,1,9/26/2022,18,B739,PK-LGR,LNI276,5.818741,233.449091
1,2,9/26/2022,36,B739,PK-LHQ,LNI140,6.300770,201.583636
2,3,9/26/2022,36,A320,PK-SJU,SJV837,8.316118,184.052308
3,4,9/26/2022,36,A320,PK-GLY,CTV938,5.656915,195.349091
4,5,9/26/2022,36,AT76,PK-WJP,WON1243,7.655067,186.145714


In [22]:
# Calculating the mean elevation for each label in the original dataframe
avg_diffs_vs = df.groupby('label')['diff_vs'].mean().reset_index()

# Merging the 'transformation' dataframe with the calculated mean elevations
transformation = pd.merge(transformation, avg_diffs_vs, on='label', how='left')

# Renaming the column to 'avg_elevation'
transformation.rename(columns={'diff_vs': 'avg_diff_vs'}, inplace=True)


transformation.head()


,label,tanggal,heading,aircraft_code,registration,callsign,avg_elevation,avg_altitude_m,avg_diff_vs
0,1,9/26/2022,18,B739,PK-LGR,LNI276,5.818741,233.449091,46.545455
1,2,9/26/2022,36,B739,PK-LHQ,LNI140,6.300770,201.583636,81.454545
2,3,9/26/2022,36,A320,PK-SJU,SJV837,8.316118,184.052308,157.538462
3,4,9/26/2022,36,A320,PK-GLY,CTV938,5.656915,195.349091,110.545455
4,5,9/26/2022,36,AT76,PK-WJP,WON1243,7.655067,186.145714,36.571429


In [24]:
# transformation.to_csv('labelBaru_II.csv')